In [26]:
import sys
sys.path.append('..')
from transformers import AutoModelForMaskedLM, AutoTokenizer
from prosst.structure.quantizer import PdbQuantizer
from Bio import SeqIO
import torch
import pandas as pd
from scipy.stats import spearmanr

In [ ]:
# import os
# os.environ["http_proxy"] = "http://127.0.0.1:15777"
# os.environ["https_proxy"] = "http://127.0.0.1:15777"

# # 

Load ProSST from Hugging Face. 
(You may need to configure the proxy settings if you are in a region that cannot access the hugging face model.)

In [ ]:
deprot = AutoModelForMaskedLM.from_pretrained("AI4Protein/ProSST-2048", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("AI4Protein/ProSST-2048", trust_remote_code=True)

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

configuration_prosst.py:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/ProSST-2048:
- configuration_prosst.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_prosst.py:   0%|          | 0.00/53.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/ProSST-2048:
- modeling_prosst.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Load strcuture quantizer

In [3]:
processor = PdbQuantizer()

In [25]:
processor

Read protein sequence

In [5]:
residue_sequence = str(SeqIO.read('example_data/GRB2_HUMAN_Faure_2021.fasta', 'fasta').seq)
    

In [14]:
len(residue_sequence)

217

Quantize the structure

In [6]:
structure_sequence = processor("example_data/GRB2_HUMAN_Faure_2021.pdb")

Processing PDB files: 100%|██████████| 1/1 [00:02<00:00,  2.55s/it]


Shift the quantized structure sequence, (for 3 special tokens [CLS], [SEP] and [PAD])

In [13]:
len(structure_sequence['2048']['GRB2_HUMAN_Faure_2021.pdb']['struct'])

217

In [27]:
structure_sequence

{'2048': {'GRB2_HUMAN_Faure_2021.pdb': {'struct': [1583,
    23,
    1525,
    1671,
    399,
    1985,
    1671,
    771,
    1985,
    684,
    1890,
    1359,
    1751,
    213,
    2,
    1985,
    229,
    771,
    1985,
    1985,
    1671,
    23,
    1196,
    1985,
    1985,
    1985,
    471,
    229,
    1359,
    356,
    796,
    1890,
    23,
    1523,
    1468,
    1289,
    23,
    1985,
    471,
    23,
    2,
    785,
    796,
    1890,
    23,
    1985,
    1985,
    1985,
    1890,
    922,
    23,
    922,
    1583,
    1890,
    1671,
    229,
    356,
    356,
    1289,
    1289,
    356,
    1289,
    2,
    1289,
    78,
    773,
    621,
    661,
    661,
    1468,
    621,
    661,
    972,
    1275,
    1468,
    55,
    1468,
    1468,
    1975,
    922,
    922,
    922,
    23,
    1975,
    922,
    922,
    1289,
    1289,
    356,
    1523,
    23,
    1890,
    403,
    922,
    1583,
    1975,
    1289,
    1985,
    1671,
    1975,
    1975,
    338,

In [ ]:
# structure_sequence_offset = [i + 3 for i in structure_sequence]
structure_sequence_offset = structure_sequence['2048']['GRB2_HUMAN_Faure_2021.pdb']['struct']

Prepare model input

In [19]:
tokenized_res = tokenizer([residue_sequence], return_tensors='pt')
input_ids = tokenized_res['input_ids']
attention_mask = tokenized_res['attention_mask']
structure_input_ids = torch.tensor([1, *structure_sequence_offset, 2], dtype=torch.long).unsqueeze(0)

In [34]:
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])

Inferece 

In [20]:
with torch.no_grad():
    outputs = deprot(
        input_ids=input_ids,
        attention_mask=attention_mask,
        ss_input_ids=structure_input_ids
    )
logits = torch.log_softmax(outputs.logits[:, 1:-1], dim=-1).squeeze()

In [37]:
outputs.logits.size()

torch.Size([1, 219, 25])

Score mutants

In [21]:
df = pd.read_csv("example_data/GRB2_HUMAN_Faure_2021.csv")
mutants = df['mutant'].tolist()

In [38]:
mutants

['T159M:D166V',
 'T159F:G203C',
 'T159F:G203A',
 'T159F:G200V',
 'T159F:G200T',
 'T159F:G200R',
 'T159F:G200K',
 'T159F:G200A',
 'T159F:G196V',
 'T159F:G196R',
 'T159F:G196L',
 'T159F:G180D',
 'T159F:G180A',
 'T159F:G176W',
 'T159F:G203I',
 'T159F:G203K',
 'T159F:G203L',
 'T159F:G203M',
 'T159F:M186E',
 'T159F:M186C',
 'T159F:G203D',
 'T159F:K195V',
 'T159F:G203E',
 'T159E:Y209F',
 'T159E:P169R',
 'T159E:Q201V',
 'T159E:Q201S',
 'T159E:Q201P',
 'T159E:Q201N',
 'T159E:Q201M',
 'T159E:Q201L',
 'T159E:Q201K',
 'T159E:R178A',
 'T159E:Q201H',
 'T159E:Q201F',
 'T159E:Q201E',
 'T159E:Q201A',
 'T159E:Q170W',
 'T159E:P212N',
 'T159E:P206G',
 'T159E:P169T',
 'T159E:Q201G',
 'T159E:W193R',
 'T159E:Y209G',
 'T159E:P169D',
 'T159F:K195R',
 'T159F:K195A',
 'T159F:C198A',
 'T159F:A197V',
 'T159F',
 'T159E:Y209V',
 'T159E:Y209R',
 'T159E:Y209N',
 'T159E:Y209L',
 'T159F:C198G',
 'T159E:M204N',
 'T159F:E174M',
 'T159F:F165G',
 'T159F:F205V',
 'T159F:F205S',
 'T159F:F205R',
 'T159F:F205N',
 'T159F:F205M'

In [22]:
vocab = tokenizer.get_vocab()
pred_scores = []
for mutant in mutants:
    mutant_score = 0
    for sub_mutant in mutant.split(":"):
        wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
        pred = logits[idx, vocab[mt]] - logits[idx, vocab[wt]]
        mutant_score += pred.item()
    pred_scores.append(mutant_score)

In [39]:
vocab

{'<pad>': 0,
 '<cls>': 1,
 '<eos>': 2,
 'A': 3,
 'C': 4,
 'D': 5,
 'E': 6,
 'F': 7,
 'G': 8,
 'H': 9,
 'I': 10,
 'K': 11,
 'L': 12,
 'M': 13,
 'N': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'V': 20,
 'W': 21,
 'Y': 22,
 '<unk>': 23,
 '<mask>': 24}

Compute the spearman correlation

In [44]:
len(pred_scores)

63366

In [45]:
df['DMS_score']

0       -0.923700
1       -0.811372
2       -0.959640
3       -0.979989
4       -0.855926
           ...   
63361    0.050346
63362   -0.204821
63363   -0.152771
63364   -0.136535
63365    0.078846
Name: DMS_score, Length: 63366, dtype: float64

In [24]:
spearmanr(pred_scores, df['DMS_score'])

SignificanceResult(statistic=0.34666187774536855, pvalue=0.0)